In [ ]:
using Plots
using LinearAlgebra
using Printf
using LaTeXStrings
using DifferentialEquations
using FFTW

In [ ]:
default(lw=2, markersize=6,
    xtickfont=font(12), ytickfont=font(12),
    guidefont=font(14), legendfont=font(12), titlefont=font(12))

# Solve a BVP
$$
u - u'' = f(x)
$$
on $(0,2\pi)$ with periodic boundary conditions.  Solve this spectrally and using the collocation method for comparison.


In [ ]:
@show n = 2^4; # number of points
x = LinRange(0, 2 * π, n + 1)[1:end-1];
dx = im * [0:1:n÷2; -n÷2+1:1:-1];


In [ ]:

f(x) = x;

F = fft(f.(x));
U = @. F / (1 - dx^2);
u = ifft(U);


In [ ]:
plot(x, real.(u))

In [ ]:
function bvp_rhs!(du, u, p, t)
    du[1] = u[2]
    du[2] = u[1] - f(t)
end

function bvp_bc!(res, u, p, t)
    res[1] = u[1][1] - u[end][1] # u(0) = u(2*π)
    res[2] = u[1][2] - u[end][2] # u'(0) = u'(2*π)
end

In [ ]:
tspan = (0, 2*π)
# [0., 0.] is an initial (constant) guess for the solution of the bvp
bvp = BVProblem(bvp_rhs!, bvp_bc!, [0.0, 0.0], tspan)


In [ ]:
# solve with 4th order collocation method, must specify an initial mesh spacing
soln = solve(bvp, MIRK4(), dt=0.1)


In [ ]:
scatter(x, real.(u), label="FFT")
plot!(soln, idxs=(1), label="Collocation")
xlabel!(L"x")
ylabel!(L"u")